In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

'''
    Author:	 Matthew Ogden and Graham West
    Created:	01 Sep 2020
Description:	This code is a modified version of code written by Graham West at https://github.com/gtw2i/GA-Galaxy. 
                The pupose of this code is to genetrically modify the SPAM orbital parameters for galactic models.
'''

# Add parent directory
from sys import path as sysPath
from os import listdir
sysPath.append('../')

# Add custom modules
sysPath.append('../Machine_Score/')

from Support_Code import general_module as gm
from Support_Code import info_module as im
from Genetic_Algorithm import main_Genetic_Algorithm as ga
import main_SIMR as simr

# General modules
from os import path
import numpy as np
import numpy.linalg as LA
import pandas as pd
from matplotlib import pyplot as plt
import random
import pickle

# Print testing
gm.test()
im.test()
ga.test()
simr.test()


GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew's information module for SPAM
GA: Hi!  You're in Matthew's main code for all things genetric algorithm.
SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions


In [2]:
tDir = '../targetDir'
#print('Before: \'%s\' \nAfter: \'%s\'' % (tDir,gm.validPath(tDir) ))
tDir = gm.validPath(tDir)
rInfo = None

tInfo = im.target_info_class( targetDir = tDir, printAll=False)
if tInfo.status == False:
    print("WARNING")
else:
    print("Target Good!")
    im.tabprint("Grabbing first model.")
    rInfo = tInfo.getRunInfo()
    #print(rInfo)
    if type(rInfo) == type(None):
        print("BAD RUN!!!!")
    else:
        print("RUN GOOOD!!!")
        im.tabprint('rInfo status: %s' % (rInfo.status) )

Target Good!
	 - Grabbing first model.
RUN GOOOD!!!
	 - rInfo status: True


## Working Graham's Code

In [3]:
cmdArgs = gm.inArgClass()
cmdArgs.setArg('paramLoc', '../param/test_param.json')

tInfo = simr.target_initialize( cmdArgs, tInfo )
runArgs = simr.target_prep_cmd_params( tInfo, cmdArgs )
#runArgs.printArg()

runArgs.printBase = False
runArgs.printAll = False
runArgs.newAll = True
runArgs.newInfo = True
runArgs.rmRunDir = False
runArgs.overWrite = True
runArgs.overwrite = True


SIMR.target_initialize:
	 - tDir: None
	 - tInfo: <class 'Support_Code.info_module.target_info_class'>
	 - True - 587722984435351614 - 1293 Models
SIMR: target_prep_cmd_params: 587722984435351614


In [4]:
ga_param = gm.readJson( '../param/init_ga_param.json' )

ga_param['generation_number'] = int( ga_param['generation_total_number'] \
                                    / ga_param['phase_number'] )
ga_param['covariance_burn'] = np.power( ga_param['generation_number']/2, -1 )
ga_param['covariance_mix_probability'] = ga_param['covariance_mix_probability']/np.sum(ga_param['covariance_mix_probability'])

ga_param['parameter_limits'] = np.array( ga_param['parameter_limits'] )

gm.pprint( ga_param )

{'covariance_burn': 2.0,
 'covariance_mix_amplitude': [0.3333333333333333,
                              3.0],
 'covariance_mix_matrix': True,
 'covariance_mix_probability': array([0.33333333, 0.33333333, 0.33333333]),
 'covariance_scale': 0.05,
 'generation_number': 1,
 'generation_total_number': 2,
 'parameter_fixed_values': [-9.938529968261719,
                            -4.58050012588501,
                            3.2737700939178467,
                            0.2912585437297821,
                            258.48944091796875,
                            -23.275182723999023,
                            0.4857964813709259,
                            48.02430725097656,
                            9.125129699707031,
                            17.649972915649414,
                            129.69229125976562,
                            36.8125,
                            -41.784698486328125,
                            51.42856979370117],
 'parameter_limits': array([[ -9.93852

In [5]:

tmp = '''
# Creation and saving of needed variables for genetic algorythm. 
ga_param = {}

# Total number of generations to complete in a GA experiment.
ga_param['generation_total_number'] = 2

# Number of phases during a complete GA experiment. 
nPhase = 2
ga_param['phase_number'] = 2

# Number of generations during a single phase. 
nGen = 1
ga_param['generation_number'] = int( ga_param['generation_total_number'] \
                                    / ga_param['phase_number'] )

# number of SPAM / GA parameters per model.
ga_param['parameter_number'] = 14 

# total list of indices for parameters that will change/mutate during GA to fit target. 
ga_param['parameter_to_fit'] = [ 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13 ]

# SPAM / GA parameter limits
ga_param['parameter_limits'] = xLim

# Target / fixed parameters
ga_param['parameter_fixed_values'] = pReal

# Size of model population during each generational step.
nPop = 2**2	# size of population at each step
ga_param['population_size'] = 2**2

# sets of parameters to change during seperate phases.
ga_param['phase_parameter_to_fit'] = []
ga_param['phase_parameter_to_fit'].append([ 4, 5, 10, 11, 12, 13 ])
ga_param['phase_parameter_to_fit'].append([ 2, 3, 6, 8, 9 ])

# Set population initialization method.
ga_param['population_initialization_method'] = 1

# Number of loops to find a maximum correlation.
ga_param['population_sample_size'] = 1000

# Psi value....
ga_param['parameter_psi'] = [ 1, 1 ]

# ga parameter initialization
ga_param['covariance_mix_matrix'] = True
ga_param['covariance_mix_amplitude'] = [ 1.0/3.0, 3.0 ]
ga_param['covariance_scale'] = 0.05
ga_param['covariance_mix_probability'] = np.array( [ 1.0, 1.0, 1.0 ] )
ga_param['population_keep'] = 1
ga_param['population_reseed_ratio'] = 0.125

# Self creating/editing variables for GA experiment
ga_param['covariance_burn'] = np.power( ga_param['generation_number']/2, -1 )
ga_param['covariance_mix_probability'] = ga_param['covariance_mix_probability']/np.sum(ga_param['covariance_mix_probability'])


gm.saveJson( ga_param, '../param/init_ga_param.json', pretty = True, convert_numpy_array = True )

print('done')
'''

In [6]:

class ga_machine_score_wrapper:
    
    runArgs = None # arguments while scoring individual models
    tInfo   = None # class for target system
    psi     = None # variable needed for converting from ga to spam parameters
    prog    = None # variable for printing scoring progress
    
    def __init__( self, runArgs, tInfo, psi, printProg ):
        
        # Save variables
        self.runArgs = runArgs
        self.tInfo   = tInfo
        self.psi     = psi
        self.prog    = printProg
        
        if self.prog:  print("SIMR.ga_machine_score_wrapper")
        
    def score_models( self, ga_params ):
        spam_parameters = ga.convert_ga_to_spam( ga_params, self.psi )
        scores = simr.score_models_iter( self.tInfo, self.runArgs, \
                                        spam_parameters, printProg = self.prog )
        return scores

# End ga_machine score wrapper

scorer = ga_machine_score_wrapper( runArgs, tInfo, ga_param['parameter_psi'], True )


SIMR.ga_machine_score_wrapper


In [7]:
from Genetic_Algorithm.main_Genetic_Algorithm import *

In [8]:
outDir = gm.validPath ( '../tmpDir/' )
outBase = outDir + 'Results_%s_' % gm.getFileFriendlyDateTime()

exp_details = {}

exp_details['name'] = 'Genetic Algorithm Creation'
exp_details['info'] = 'This file is the prototype for creating \
    the Genetic Algorythm pipeline.'
exp_details['score_parameters'] = runArgs.get('scoreParams')
exp_details['ga_paramters'] = ga_param

print( exp_details.keys() )

eLoc = outBase + 'Details.json'
print(eLoc)

#gm.saveJson( exp_details, eLoc, pretty = True, convert_numpy_array=True )
gm.saveJson( exp_details, eLoc, pretty = False, convert_numpy_array=True )

print("Saved")

dict_keys(['name', 'info', 'score_parameters', 'ga_paramters'])
/mnt/c/Users/ogden/OneDrive/Documents/_Galaxy_Code/galaxyJSPAM/tmpDir/Results_2022-04-28T13-52-25_Details.json
Saved


In [9]:

    
Genetic_Algorithm_Experiment( ga_param, scorer.score_models, outBase, True )

GA.Genetic_Algorithm_Experiment: Beginning Experiment
Master: Received 3 / 4 - 0.6408727264231049
Master: Complete
GA: step: 0 / 1
Master: Received 3 / 4 - 0.6317319129116885
Master: Complete


NameError: name 'pickle' is not defined

In [ ]:
tmp = '''
pFile = '../Genetic_Algorithm/587722984435351614_combined.txt'
print( path.exists(pFile), pFile )

zooThresh = 0.5

import math
from copy import deepcopy
data, psi, nModel, nCol = ga.ReadAndCleanupData( pFile, zooThresh )
print("data: ", data.shape)
print("PSI: ", psi.shape)
print("nModel: ", nModel)
print("nCol: ", nCol)
#print(psi)
#####################
###   VARIABLES   ###
#####################

# target ID
zooBase = "587722984435351614"
targetInd = 0
nParam    = 14		# number of SPAM parameters
shrink   = 1.0		# fraction of max xLim

pReal   = data[targetInd,0:-1]
psiReal = psi[targetInd,:]
print('psiReal', psiReal)

# read max xLim and window bounds
#XXX = np.loadtxt( "../Genetic_Algorithm/Bounds_587722984435351614.txt" )
#	mmm = XXX[:nParam,:]

# get parameter stats
mins = np.min( data, axis=0 )[0:-1]
maxs = np.max( data, axis=0 )[0:-1]
mmm = [ mins, maxs ]
mmm = np.transpose(np.array(mmm))
#	print mmm
# modify max xLim
#	mmm[2,:]  = np.array([ -10.0,  10.0 ])
#	mmm[2,:]  = np.array([   0.0,   5.0 ])
mmm[2,:]  = np.array([   0.0,   mmm[2,1] ])  # sym
"""
mmm[3,:]  = np.array([  -0.9,   0.9 ])
mmm[4,:]  = np.array([   0.0, 360.0 ])
mmm[5,:]  = np.array([ -90.0,  90.0 ])
#	mmm[6,:]  = np.array([   0.2,   0.8 ])
#	mmm[7,:]  = np.array([  10.0,  70.0 ])
#	mmm[8,:]  = np.array([   0.5,  10.0 ])
#	mmm[9,:]  = np.array([   0.5,  10.0 ])
"""
mmm[10,:] = np.array([   0.0, 180.0 ])	# sym
mmm[11,:] = np.array([   0.0, 180.0 ])	# sym
mmm[12,:] = np.array([ -89.0,  89.0 ])	
mmm[13,:] = np.array([ -89.0,  89.0 ])	

# shrink xLim
xLim = np.zeros((nParam,2))
for i in range(nParam):
    xLim[i,0] = pReal[i] - shrink*(pReal[i] - mmm[i,0])
    xLim[i,1] = shrink*(mmm[i,1] - pReal[i]) + pReal[i]
# end

print("    min      real     max")
for i in range(nParam):
#		print xLim[i,0], pReal[i], xLim[i,1]#, pWidth[i]
    print( "{:8.2f} ".format(xLim[i,0]) + "{:8.2f} ".format(pReal[i]) + "{:8.2f} ".format(xLim[i,1]) )
# end
print(" ")
'''